In [1]:
import time
import requests
import json
import pickle
import openai

In [2]:
# OpenAI API credentials
AZURE_OPENAI_KEY = "41d471bedea9412e9d0a5461cf69bfc8"
AZURE_OPENAI_ENDPOINT = "https://openai-resource-team-6-france.openai.azure.com/"
AZURE_ENGINE_NAME = "gpt35-team-6"

openai.api_type = "azure"
openai.api_base = AZURE_OPENAI_ENDPOINT
openai.api_version = "2023-05-15"
openai.api_key = AZURE_OPENAI_KEY

In [3]:
'''
'Once upon a time, in a cozy little town called Sunnyville, there lived a young girl named Lily. Lily loved playing outside, chasing butterflies, and exploring the colorful meadows. But there was one thing that made Lily\'s adventures a little tricky – she had allergies.\n\nAllergies are like mischievous little creatures that live inside our bodies. Lily\'s body had some special creatures called "allergens" that made her feel sick. These allergens could be found in things like pollen from flowers, fur from cuddly animals, or even certain foods.\n\nOne sunny morning, Lily woke up with a stuffy nose and itchy, watery eyes. She knew her allergies were acting up again. Her mom noticed and said, "Oh, dear, it seems like your allergies are bothering you today. Let\'s figure out what\'s causing it."\n\nLily and her mom decided to go on an adventure together to find the culprit of her allergies. They walked through the flower-filled meadow, where Lily would often play. Suddenly, Lily sneezed! "Achoo!" It was the flowers! Lily\'s body didn\'t get along well with the pollen they released into the air.\n\nAs they continued their adventure, they met a friendly dog named Max. He wagged his tail happily, excited to meet Lily. But as Lily got closer to Max, she started to itch all over. It was Max\'s fur! Lily\'s body didn\'t like the tiny particles that clung to the dog\'s fur.\n\nLily\'s mom explained that allergies were like Lily\'s body having very picky taste buds. Just as some people might not like the taste of certain foods, Lily\'s body didn\'t like the pollen or Max\'s fur. Even though these things might be harmless to others, Lily\'s body saw them as intruders it needed to protect her from.\n\nBut Lily wasn\'t discouraged. She learned that she could still have fantastic adventures without letting her allergies ruin the fun. She started wearing a cool hat and sunglasses to shield her eyes from pollen. And whenever she encountered animals like Max, she made sure to wash her hands and clothes afterward to get rid of any allergens.\n\nLily also discovered that some foods made her feel itchy and uncomfortable. She learned to read food labels and avoid those ingredients. But there were still plenty of delicious meals and snacks she could enjoy without any problems.\n\nAs Lily embraced these new strategies, she found she could still play outside, meet new friends, and have amazing adventures. Her allergies didn\'t hold her back anymore. And whenever she needed a break, she would curl up with a good book and travel to imaginary worlds where allergies didn\'t exist.\n\nThe end.'
'''

'\n\'Once upon a time, in a cozy little town called Sunnyville, there lived a young girl named Lily. Lily loved playing outside, chasing butterflies, and exploring the colorful meadows. But there was one thing that made Lily\'s adventures a little tricky – she had allergies.\n\nAllergies are like mischievous little creatures that live inside our bodies. Lily\'s body had some special creatures called "allergens" that made her feel sick. These allergens could be found in things like pollen from flowers, fur from cuddly animals, or even certain foods.\n\nOne sunny morning, Lily woke up with a stuffy nose and itchy, watery eyes. She knew her allergies were acting up again. Her mom noticed and said, "Oh, dear, it seems like your allergies are bothering you today. Let\'s figure out what\'s causing it."\n\nLily and her mom decided to go on an adventure together to find the culprit of her allergies. They walked through the flower-filled meadow, where Lily would often play. Suddenly, Lily sneez

In [4]:
def ask_gpt(command, text):
    time.sleep(1) # to avoid Azure API throttling
    content = openai.ChatCompletion.create(
        engine=AZURE_ENGINE_NAME,
        messages=[
            {"role": "system", "content": command},
            {"role": "user", "content": text}
        ]
    )
    return content.choices[0].message["content"]

def create_story(request):
    command = "You are a helpful storyteller for children."
    return ask_gpt(command, request)

def get_summary(story):
    command = "You will be provided with a story, and your task is to summarize the story using at most 70 words."
    return ask_gpt(command, story)

def get_title(story, attempt=0):
    command = "You will be provided with a story, and your task is to create a title for it using at most 15 words."
    title = ask_gpt(command, story)
    num_words = len(title.split())
    if (num_words > 15 or num_words==0):
        if attempt < 3:
            return get_title(story, attempt+1)
        else:
            return "Once upon a time..."
    return title

def get_feeling(story, attempt=0):
    command = "You will be provided with a short story. Your task is to summarize the story using one word that describes the overall emotion " +\
        "of the paragraph. For example, if the story is about a happy event, you might use the word joy to summarize it. You should use one of " +\
        "the following words: happiness, sadness, anger, fear, surprise, disgust, contempt, resilience."
    feeling = ask_gpt(command, story).lower()
    if feeling not in ["happiness", "sadness", "anger", "fear", "surprise", "disgust", "contempt", "resilience", "neutral"]:
        if attempt < 3:
            return get_feeling(story, attempt+1)
        else:
            return "neutral"
    return feeling

def get_sentiment_level(story, attempt=0):
    command = "You will be provided with a short story. Your task is to get the sentiment level of the overall paragraph. " +\
        "You should use one of the following words: very positive, positive, neutral, negative, very negative."
    sentiment = ask_gpt(command, story).lower()
    if sentiment not in ["very positive", "positive", "neutral", "negative", "very negative"]:
        if attempt < 3:
            return get_sentiment_level(story, attempt+1)
        else:
            return "neutral"
    return sentiment

def get_tags(story, attempt=0):
    command = "You will be provided with a short story. Your task is to get the category of the overall paragraph. You should use one of" +\
        " the following words: action, adventure, animation, biography, comedy, crime, documentary, drama, family, fantasy, history," +\
        " horror, music, musical, mystery, romance, sci-fi, short, sport, thriller, war, western."
    tags = ask_gpt(command, story).lower()
    if tags not in ["action", "adventure", "animation", "biography", "comedy", "crime", "documentary", "drama", "family", "fantasy", "history",
        "horror", "music", "musical", "mystery", "romance", "sci-fi", "short", "sport", "thriller", "war", "western"]:
        if attempt < 3:
            return get_tags(story, attempt+1)
        else:
            return "drama"
    return tags            

def get_paragraphs(story):
    # split output_chatgpt into paragraphs
    paragraphs = story.split('\n\n')
    paragraphs = [x for x in paragraphs if x != '']
    # remove spaces from the beginning of each paragraph
    paragraphs = [x[1:] if x[0] == ' ' else x for x in paragraphs]
    return paragraphs

def get_scenes(story):
    scenes = []

    paragraphs = get_paragraphs(story)

    for idx, paragraph in enumerate(paragraphs):
        sentiment_level = get_sentiment_level(paragraph)
        feeling = get_feeling(paragraph)

        scene = {
            "id": str(idx),
            "text": paragraph,
            "sentiment": sentiment_level,
            "polarity": feeling,
            "tone_song": None,
            "image": None
        }

        scenes.append(scene)

    return scenes

In [5]:
# For this case, the main_id is the timestamp when the story is created
main_id = str(int(time.time()))
main_id

'1689788440'

In [6]:
story = "Once upon a time, there was a little boy named Timmy. Timmy was a happy and a playful boy who loved going out and playing with his friends. However, Timmy had some allergies which made it a little difficult for him to enjoy certain things like his friends did.\n\nOne day, Timmy went on a picnic with his friends. His mother had carefully packed him a lunchbox full of delicious food that she knew he could eat without any problems. As the children were playing and enjoying their food, Timmy noticed that his nose started to itch, and his eyes began to water. Suddenly, he started to feel unwell, and his throat began to hurt.\n\nTimmy's mother was nearby and noticed he wasn't feeling well. She quickly realized that Timmy was having an allergic reaction to something he had eaten. She gave him some medicine and rushed him to the hospital. Luckily, they were able to treat him before things got worse.\n\nAfterward, Timmy's mother explained to him about allergies. She told him that sometimes our bodies don't like certain things, and when we come into contact with them, it can cause us to feel unwell. She told him that it was important to always be careful with what he ate and to let others know about his allergies so they could help keep him safe.\n\nFrom that day on, Timmy was very careful about what he ate and always let his friends and family know about his allergies. He knew that being cautious was better than risking another scary reaction. With his mother's help and the knowledge she provided him, Timmy was able to have more fun and enjoy being with his friends without worrying about his allergies."

In [7]:
request = "Can you explain allergies to me while telling me a children's story?"
#story = create_story(request)
print(story)

main = {
    "id": main_id,
    "title": get_title(story),
    "summary": get_summary(story),
    "main_prompt": request,
    "tone": get_feeling(story),
    "tags": get_tags(story),
    "history": get_scenes(story)
}


Once upon a time, there was a little boy named Timmy. Timmy was a happy and a playful boy who loved going out and playing with his friends. However, Timmy had some allergies which made it a little difficult for him to enjoy certain things like his friends did.

One day, Timmy went on a picnic with his friends. His mother had carefully packed him a lunchbox full of delicious food that she knew he could eat without any problems. As the children were playing and enjoying their food, Timmy noticed that his nose started to itch, and his eyes began to water. Suddenly, he started to feel unwell, and his throat began to hurt.

Timmy's mother was nearby and noticed he wasn't feeling well. She quickly realized that Timmy was having an allergic reaction to something he had eaten. She gave him some medicine and rushed him to the hospital. Luckily, they were able to treat him before things got worse.

Afterward, Timmy's mother explained to him about allergies. She told him that sometimes our bodies

# FALTAR:
- Prompts para gerar as imagens
- adicionar o tempo que demorou para gerar tudo

In [8]:
main

{'id': '1689788440',
 'title': 'Allergic Adventures: A Tale of Caution and Confidence',
 'summary': 'Timmy, a playful little boy, enjoyed playing with his friends but had allergies that made it challenging. During a picnic, Timmy had an allergic reaction but his mother quickly treated him and took him to the hospital. She explained allergies to him, emphasizing the importance of being cautious and informing others. With this knowledge, Timmy enjoyed his time with friends without worry.',
 'main_prompt': "Can you explain allergies to me while telling me a children's story?",
 'tone': 'resilience',
 'tags': 'family',
 'history': [{'id': '0',
   'text': 'Once upon a time, there was a little boy named Timmy. Timmy was a happy and a playful boy who loved going out and playing with his friends. However, Timmy had some allergies which made it a little difficult for him to enjoy certain things like his friends did.',
   'sentiment': 'neutral',
   'polarity': 'resilience',
   'tone_song': None,

In [9]:
# convert to pickle
with open(f'../../stories/{main_id}.pkl', 'wb') as f:
    pickle.dump(main, f)

In [10]:
# convert to json
main_json = json.dumps(main)
# save to file
with open(f'../../stories/{main_id}.json', 'w') as f:
    f.write(main_json)

In [11]:
'''
Main
{
	"id": "String",
	"title": "String",
    "summary": "String",
	"main_prompt": "String*", # prompt that generated the main story
	"tone": "String", # happy, sad, etc.
    tags: ["String"], # comedy, drama, etc.
	"history": [Scene]
}
Scene
{
	"id": "String",
	"text": "String",
	"sentiment": "String",
	"polarity": "String", # negative to positive
	"tone_song": "String",
	"image": "b64"
}
'''


'\nMain\n{\n\t"id": "String",\n\t"title": "String",\n    "summary": "String",\n\t"main_prompt": "String*", # prompt that generated the main story\n\t"tone": "String", # happy, sad, etc.\n    tags: ["String"], # comedy, drama, etc.\n\t"history": [Scene]\n}\nScene\n{\n\t"id": "String",\n\t"text": "String",\n\t"sentiment": "String",\n\t"polarity": "String", # negative to positive\n\t"tone_song": "String",\n\t"image": "b64"\n}\n'